In [1]:

!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

zsh:1: command not found: wget
unzip:  cannot find or open glove.6B.zip, glove.6B.zip.zip or glove.6B.zip.ZIP.


In [2]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

NameError: name 'nltk' is not defined

In [ ]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.preprocessing import MultiLabelBinarizer
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn import preprocessing
from nltk.tokenize import TreebankWordTokenizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import *
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.metrics import plot_confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import *
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
import seaborn as sn

In [ ]:
path = "data"
stop_words = stopwords.words("english")

In [ ]:
train = pd.read_csv(path+'/DBLPTrainset.csv')
test_labels =  pd.read_csv(path+'/DBLPTestGroundTruth.csv')
test_articles = pd.read_csv(path+'/DBLPTestset.csv')
len(set(train.Label))

In [ ]:
def lemma_tokenizer(text):
  tokens = TreebankWordTokenizer().tokenize(text)
  tokens = [word.lower() for word in tokens]
  filtered_words = [word for word in tokens if word not in stop_words]
  lemmatizer = WordNetLemmatizer()
  lemmas = [lemmatizer.lemmatize(word) for word in filtered_words]
  return lemmas

In [ ]:
def stem_tokenizer(text):
      tokens = nltk.word_tokenize(text)
      tokens = [word.lower() for word in tokens if word.isalpha()]
      filtered_words = [word for word in tokens if word not in stop_words]
      stemmer = PorterStemmer()
      porter_tokens = [stemmer.stem(word) for word in filtered_words]
      return porter_tokens

In [ ]:
def count_vectorizer(train_text, test_text, tokenizer=lemma_tokenizer, ngram=(2,2)):
    if tokenizer:
        vectorizer = CountVectorizer(ngram_range=ngram, tokenizer = tokenizer )
    else:
        vectorizer = CountVectorizer(ngram_range=ngram)
    vectorizer.fit(train_text)
    vectorised_train_documents_1 = vectorizer.transform(train_text)
    vectorised_test_documents_1 = vectorizer.transform(test_text)
    return vectorised_train_documents_1, vectorised_test_documents_1

In [ ]:
def tfidf_vectorizer(train_vecs, test_vecs, tokenizer=lemma_tokenizer,):
    tfidf_transformer = TfidfTransformer()
    tfidf_transformer.fit(train_vecs)
    vectorised_train_documents_1 = tfidf_transformer.transform(train_vecs)
    vectorised_test_documents_1 = tfidf_transformer.transform(test_vecs)
    return vectorised_train_documents_1, vectorised_test_documents_1

In [ ]:
best_accuracy = 0
best_combination = {}
for tokenizer in (stem_tokenizer, lemma_tokenizer, None):
    for ngram in ((1,1), (1,2),(1,3),(2,2)):
        X_train, X_test =  count_vectorizer(train.Title, test_articles.Title, tokenizer, ngram)
        for classifier in (SGDClassifier, MultinomialNB):
            for tfidf in (True, False):
                if tfidf:
                    X_train, X_test =  tfidf_vectorizer(X_train, X_test)
                predictor = classifier()
                predictor.fit(X_train, train.Label)
                predictions_1 = predictor.predict(X_test)
                current_accuracy = accuracy_score(test_labels.Label, predictions_1)
                print(str(tokenizer)+', '+str(classifier)+', ' +str(tfidf)+', '+str(ngram))
                print(current_accuracy)
                if current_accuracy > best_accuracy:
                    best_accuracy =  current_accuracy
                    best_combination =  {'tokenizer':tokenizer, 'classifier':classifier, 'tfidf':tfidf, 'ngram':ngram}

In [ ]:
print(best_accuracy)
print(best_combination)

In [ ]:
X_train, X_test =  count_vectorizer(train.Title, test_articles.Title, best_combination['tokenizer'], best_combination['ngram'])
if best_combination['tfidf']:
    X_train, X_test =  tfidf_vectorizer(X_train, X_test)
predictor = best_combination['classifier']()
predictor.fit(X_train, train.Label)
predictions_1 = predictor.predict(X_test)

In [ ]:
confusion_matrix = confusion_matrix(test_labels.Label, predictions_1)

In [ ]:
df_cm = pd.DataFrame(confusion_matrix, index = [dictionary[i] for i in range(5)],
                  columns = [dictionary[i] for i in range(5)])
plt.figure(figsize = (5,5))
sn.heatmap(df_cm, annot=True)

In [ ]:
print(classification_report(test_labels.Label, predictions_1, target_names=list(labels)))

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [ ]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

vectorizer = TextVectorization(max_tokens=20000, output_sequence_length=200)
text_ds = tf.data.Dataset.from_tensor_slices(train.Title).batch(128)
vectorizer.adapt(text_ds)

In [ ]:
voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

In [ ]:
path_to_glove_file = "/content/drive/MyDrive/Classification/glove.6B.100d.txt"

embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

In [ ]:
num_tokens = len(voc) + 2
embedding_dim = 100
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

In [ ]:
from tensorflow.keras.layers import Embedding

embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
)

In [ ]:
from tensorflow.keras import layers

int_sequences_input = keras.Input(shape=(None,), dtype="int64")
embedded_sequences = embedding_layer(int_sequences_input)
x = layers.Conv1D(128, 5, activation="relu")(embedded_sequences)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu")(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu")(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)
preds = layers.Dense(len(set(train.Label)), activation="softmax")(x)
model = keras.Model(int_sequences_input, preds)
embeding_model = keras.Model(int_sequences_input, embedded_sequences)
model.summary()

In [ ]:
from sklearn.preprocessing import LabelEncoder
x_train = vectorizer(np.array([[s] for s in train.Title])).numpy()
labelencoder = LabelEncoder()
y = labelencoder.fit_transform(train.Label)
y_train =[]
for i in y:
    label = [0,0,0,0,0]
    label[i] = 1
    y_train.append(label)
y_train = np.array(y_train)

In [ ]:
print(x_train.shape)
print(y_train.shape)
print(y_train)

In [ ]:
model.compile(
    loss="categorical_crossentropy", optimizer="rmsprop", metrics=["acc"]
)
model.fit(x_train, y_train, batch_size=128, epochs=5, validation_split=0.2)

In [ ]:
x_test = vectorizer(np.array([[s] for s in test_articles.Title])).numpy()
y_test = np.array(test_labels_transformed)
model.evaluate(x_test,y_test)